# 只有画图

In [1]:
import tensorflow as tf 
import os
import scipy.io as sio
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras import utils
# itertools内置库
import itertools
import spectral
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_path = os.path.join('E:\Eric_HSI\hyperspectral_datasets')
data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))
labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))
features = data['data']
print(features.shape)
# 标签
labels = labels['groundT']
print(labels.shape)

(145, 145, 200)
(145, 145)


In [3]:
# path_y_train = r"E:\Eric_HSI\excise2\np_data\y_train" + ".npy"
# path_y_val = r"E:\Eric_HSI\excise2\np_data\y_val" + ".npy"
path_x_all = r"E:\Eric_HSI\excise2\np_data\x_all" + ".npy"
path_y_all = r"E:\Eric_HSI\excise2\np_data\y_all" + ".npy"

In [4]:
# y_train = np.load(path_y_train)
# y_val = np.load(path_y_val)
x_all= np.load(path_x_all)
y_all = np.load(path_y_all)

In [5]:
# 因为不需要进行训练，所以不用进行这项 one-hot 编码
# y_all = utils.to_categorical(y_all)

In [6]:
x_all.shape, y_all.shape

((10249, 200, 19, 19, 1), (10249,))

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv3D(32, kernel_size=(3, 3, 3), input_shape=(200, 19, 19, 1), strides=(5, 1, 1), activation='relu'),  # 17
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv3D(32, kernel_size=(3, 3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.MaxPooling3D(pool_size=2),  # 8

    tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(16, activation='softmax')]
)

In [8]:
# model.summary()

In [9]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['acc'])
model.load_weights(r'E:\Eric_HSI\excise2\weights\model_weight.ckpt')

In [10]:
# prediction = np.argmax(model.predict(x_all), axis=-1)
# np.unique(prediction)

In [11]:
height = 145
width = 145
PATCH_SIZE = 19
outputs = np.zeros((height,width))
height, width

(145, 145)

In [12]:
# image_patch=Patch(X,100,100)
# X_test_image = image_patch.reshape(1,image_patch.shape[2],image_patch.shape[0],image_patch.shape[1], 1).astype('float32')

In [13]:
labels[0][0] == 0

False

In [14]:
# calculate the predicted image
# 因为经过 padding 和 patch 操作，预测过程有一点曲折
k = 0
for i in range(height):
    for j in range(width): 
        # target = labels[i][j]
        if labels[i][j] == 0:
            continue
        else:
            prediction = np.argmax(model.predict(x_all[k].reshape(1, 200, 19, 19, 1)), axis=-1)
            k += 1
            # print(prediction)
            outputs[i][j] = prediction+1
            # print(outputs[i][j])

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv3d/Conv3D (defined at <ipython-input-14-97a1ba73254a>:10) ]] [Op:__inference_predict_function_1391]

Function call stack:
predict_function


In [ ]:
prediction = np.argmax(model.predict(x_all[1010].reshape(1, 200, 19, 19, 1)), axis=-1)
prediction

In [ ]:
np.unique(outputs)

In [ ]:
predict_image = spectral.imshow(classes=outputs.astype(int), figsize=(6, 6))

In [ ]:
data_path = os.path.join('E:\Eric_HSI\hyperspectral_datasets')
# data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))
labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))
labels = labels['groundT']
predict_image = spectral.imshow(classes=labels.astype(int), figsize=(6, 6))

In [ ]:
np.unique(labels)